In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

df = pd.concat([df_train, df_test])

In [3]:
a = df.isna().sum()
list_missing_values = [a.index[i] for i in range(len(a)) if a[a.index[i]] > 0] # features with missing values

In [4]:
for i in range(len(list_missing_values)):
    print("Part of the data missing for " + list_missing_values[i] + " : ", 100*(a[list_missing_values[i]]/len(df)))

Part of the data missing for MSZoning :  0.1370332305584104
Part of the data missing for LotFrontage :  16.649537512846866
Part of the data missing for Alley :  93.21685508735868
Part of the data missing for Utilities :  0.0685166152792052
Part of the data missing for Exterior1st :  0.0342583076396026
Part of the data missing for Exterior2nd :  0.0342583076396026
Part of the data missing for MasVnrType :  0.8221993833504625
Part of the data missing for MasVnrArea :  0.7879410757108599
Part of the data missing for BsmtQual :  2.774922918807811
Part of the data missing for BsmtCond :  2.8091812264474134
Part of the data missing for BsmtExposure :  2.8091812264474134
Part of the data missing for BsmtFinType1 :  2.7064063035286057
Part of the data missing for BsmtFinSF1 :  0.0342583076396026
Part of the data missing for BsmtFinType2 :  2.7406646111682083
Part of the data missing for BsmtFinSF2 :  0.0342583076396026
Part of the data missing for BsmtUnfSF :  0.0342583076396026
Part of the da

We can see that Alley, FireplaceQu, PoolQC, Fence and MiscFeature have more than 40% of data missing, so we'll drop these columns.

In [5]:
df = df.drop(columns=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=1)

We also have to remove the last rows with missing data :

In [6]:
df = df.dropna()

In [7]:
s = (df.dtypes == object)
categorical_cols = list(s[s].index)
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

We will use a LabelEncoder to turn our categorical features into numerical ones.

In [8]:
from sklearn.preprocessing import LabelEncoder

for column in categorical_cols:
    le = LabelEncoder()
    le.fit(df[column].astype(str))
    df[column] = le.transform(df[column].astype(str))

Correlation Matrix

In [9]:
df.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 76, dtype: int64

In [10]:
corr_abs = df.corr().abs()
s = corr_abs.unstack()
so = s.sort_values(kind="quicksort")
so

EnclosedPorch  BsmtHalfBath     0.000045
BsmtHalfBath   EnclosedPorch    0.000045
Heating        SaleType         0.000056
SaleType       Heating          0.000056
3SsnPorch      HalfBath         0.000080
                                  ...   
MoSold         Utilities             NaN
YrSold         Utilities             NaN
SaleType       Utilities             NaN
SaleCondition  Utilities             NaN
SalePrice      Utilities             NaN
Length: 5776, dtype: float64

Here, we printed the most correlated features. We can see that they are correlated at all.

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

X = df.drop('SalePrice', axis=1)
y = df.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

lr_pipeline = Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])
en_pipeline = Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])
cart_pipeline =  Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])

lr_pipeline.fit(X_train, y_train)
en_pipeline.fit(X_train, y_train)
cart_pipeline.fit(X_train, y_train)

preds_lr = lr_pipeline.predict(X_test)
preds_en = en_pipeline.predict(X_test)
preds_cart = cart_pipeline.predict(X_test)

score_lr = mean_absolute_error(y_test, preds_lr)
score_en = mean_absolute_error(y_test, preds_en)
score_cart = mean_absolute_error(y_test, preds_cart)

print('MAE of Linear Regression :', score_lr)
print('MAE of Elastic Net :', score_en)
print('MAE of Decision Tree Regression :', score_cart)

MAE of Linear Regression : 25593.99836983071
MAE of Elastic Net : 23739.20790069007
MAE of Decision Tree Regression : 28311.828467153286


In order to make some predictions, we'll use Elastic Net.